In [10]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [11]:
url_base = "http://books.toscrape.com/"
response = requests.get(url_base)

if response.status_code == 200:
    html_content = response.text
    print("Html carregado com sucesso")
else:
    print(f"Erro ao carregar html. Status: {response.status_code}")

Html carregado com sucesso


In [12]:
soup = BeautifulSoup(html_content, 'html.parser')

print(soup.prettify())

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="en-us">
 <!--<![endif]-->
 <head>
  <title>
   All products | Books to Scrape - Sandbox
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="24th Jun 2016 09:29" name="created"/>
  <meta content="" name="description"/>
  <meta content="width=device-width" name="viewport"/>
  <meta content="NOARCHIVE,NOCACHE" name="robots"/>
  <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
  <!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
  <link href="static/oscar/favicon.ico" rel="shortcut icon"/>
  <link href="static/oscar/css/styles.css" rel="stylesheet" type="tex

In [13]:
categorias_div = soup.find('div', class_='side_categories')

categorias = []

if categorias_div:
    for a_tag in categorias_div.find_all('a'):
        text = a_tag.get_text(strip=True)
        href = a_tag.get('href')
        categorias.append({'nome': text, 'link': href})

categorias.remove(categorias[0])
df = pd.DataFrame(categorias)
df.to_csv('../bases/categorias.csv', index=False, encoding='utf-8')

In [14]:
def retornar_pagina(link, pagina, categoria):
    if(pagina > 1):
        url_pagina = url_base + link.replace('index.html', f'page-{pagina}.html')
    else:
        url_pagina = url_base + link
    response_pagina = requests.get(url_pagina)
    
    if response_pagina.status_code == 200:
        html_pagina = response_pagina.text
        print(f"Página {pagina} da categoria carregada com sucesso")
    else:
        print(f"Erro ao carregar página {pagina}. Status: {response_pagina.status_code}")
        return None
    
    soup_pagina = BeautifulSoup(html_pagina, 'html.parser')

    livros = []

    for product_pod in soup_pagina.find_all('article', class_='product_pod'):
        titulo = product_pod.h3.a.get('title')
        imagem  = product_pod.find('div', class_='image_container').a.img.get('src')
        link = product_pod.h3.a.get('href')
        preco = product_pod.find('p', class_='price_color').get_text()
        disponibilidade = product_pod.find('p', class_='instock availability').get_text(strip=True)
        rating = product_pod.find('p', class_='star-rating')['class'][1]
        livros.append({'titulo': titulo, 'link': link, 'imagem': imagem, 'preco': preco, 'disponibilidade': disponibilidade, 'rating': rating, 'categoria': categoria})

    return livros

In [15]:
def salvar_categoria(nome, link):
    print(f"Salvando categoria: {nome}")
    url_categoria = url_base + link
    livros = []
    pagina = 1
    while True:
        livros_pagina = retornar_pagina(link, pagina, nome)
        if livros_pagina is None or len(livros_pagina) == 0:
            break
        livros.extend(livros_pagina)
        pagina += 1
        
    return livros
    

In [16]:
livros = []
for categoria in categorias:
    livros.extend(salvar_categoria(categoria['nome'], categoria['link']))

df_livros = pd.DataFrame(livros)
df_livros.to_csv(f'../bases/base_livros.csv', index=False, encoding='utf-8')

Salvando categoria: Travel
Página 1 da categoria carregada com sucesso
Erro ao carregar página 2. Status: 404
Salvando categoria: Mystery
Página 1 da categoria carregada com sucesso
Página 2 da categoria carregada com sucesso
Erro ao carregar página 3. Status: 404
Salvando categoria: Historical Fiction
Página 1 da categoria carregada com sucesso
Página 2 da categoria carregada com sucesso
Erro ao carregar página 3. Status: 404
Salvando categoria: Sequential Art
Página 1 da categoria carregada com sucesso
Página 2 da categoria carregada com sucesso
Página 3 da categoria carregada com sucesso
Página 4 da categoria carregada com sucesso
Erro ao carregar página 5. Status: 404
Salvando categoria: Classics
Página 1 da categoria carregada com sucesso
Erro ao carregar página 2. Status: 404
Salvando categoria: Philosophy
Página 1 da categoria carregada com sucesso
Erro ao carregar página 2. Status: 404
Salvando categoria: Romance
Página 1 da categoria carregada com sucesso
Página 2 da categoria